In [ ]:
from srai.neighbourhoods import H3Neighbourhood
from srai.regionizers import H3Regionizer
from srai.utils.geocode import geocode_to_region_gdf

In [ ]:
import geopandas as gpd
from typing import Set, Optional


def show_neighbours(
    regions_gdf: gpd.GeoDataFrame, region_id: str, neighbours_ids: Optional[Set[str]] = None
):
    regions = regions_gdf.copy()
    regions["type"] = "none"
    regions.loc[region_id, "type"] = "region"
    if neighbours_ids is not None:
        regions.loc[list(neighbours_ids), "type"] = "neighbour"
    return regions.explore(tiles="CartoDB positron", column="type")

In [ ]:
gdf_sf = geocode_to_region_gdf("Wrocław, PL")
regions_gdf = H3Regionizer(8).transform(gdf_sf)

### Region somewhere in the middle of the area under analysis

#### Direct neighbours

In [ ]:
neighbourhood_with_regions = H3Neighbourhood(regions_gdf)
region_id = "881e204089fffff"
neighbours_ids = neighbourhood_with_regions.get_neighbours(region_id)
show_neighbours(regions_gdf, region_id, neighbours_ids)

#### Only 3rd ring

In [ ]:
neighbours_ids = neighbourhood_with_regions.get_neighbours_at_distance(region_id, 3)
show_neighbours(regions_gdf, region_id, neighbours_ids)

#### Regions up to distance 3

In [ ]:
neighbours_ids = neighbourhood_with_regions.get_neighbours_up_to_distance(region_id, 3)
show_neighbours(regions_gdf, region_id, neighbours_ids)

### Edge region

The Neighbourhood only returns the indices of available regions if provided with regions_gdf

In [ ]:
edge_region_id = "881e2050bdfffff"
neighbours_ids = neighbourhood_with_regions.get_neighbours(edge_region_id)
print(neighbours_ids)

In [ ]:
show_neighbours(regions_gdf, edge_region_id, neighbours_ids)

On the other hand, an "unbiased" H3Neighbourhood returns all 6 neighbours from H3.

In [ ]:
neighbourhood_without_regions = H3Neighbourhood()
neighbourhood_without_regions.get_neighbours(edge_region_id)

### Regions with a gap

One thing to note when using the H3Neighbourhood is that it uses the h3 library under the hood. The only post-processing step is to select from among the regions under analysis. Due to that some neighbours can be included as K-th neighbours, even if there is no path of length K between them and the regions you ask for. This is because H3 still treats them as a part of the K-th ring. An example is shown below

In [ ]:
edge_region_id = "881e2050bdfffff"
neighbours_ids = neighbourhood_with_regions.get_neighbours_at_distance(edge_region_id, 2)
print(neighbours_ids)

In [ ]:
show_neighbours(regions_gdf, edge_region_id, neighbours_ids)